# Data Processing 
Apply NLP techniques such as word embeddings, stemming, lemmatization, and stop-word removal.

### Objective
An alien civilization has discovered an ancient text file on the abandoned Earth, containing
millions of sentence pairs. Their challenge is to decipher how closely related these sentence
pairs are, effectively developing a semantic similarity detection model.

### Dataset Features
- Contains ~1 million lines with paired sentences.
- Some sentence pairs share the same meaning, while others differ.
- Requires feature engineering and text preprocessing.


In [1]:
# import libraries 
import pandas as pd 
import numpy as np

In [2]:
# change this file path 
file_path = "/Users/dionnespaltman/Desktop/Luiss /Machine Learning/Project/rs2.csv"

# load the csv as a pandas dataframe 
df = pd.read_csv(file_path)

In [ ]:
# print dimensions 
print(df.shape)

(949080, 5)


In [ ]:
# display the first 5 rows
display(df.head())

,sentence1,sentence2,score,lang1,lang2
0,Ein Flugzeug hebt gerade ab.,An air plane is taking off.,5.0,de,en
1,Ein Flugzeug hebt gerade ab.,Un avión está despegando.,5.0,de,es
2,Ein Flugzeug hebt gerade ab.,Un avion est en train de décoller.,5.0,de,fr
3,Ein Flugzeug hebt gerade ab.,Un aereo sta decollando.,5.0,de,it
4,Ein Flugzeug hebt gerade ab.,飛行機が離陸します。,5.0,de,ja
